In [5]:
from keras.models import load_model  
from PIL import Image, ImageOps  
import numpy as np
import sqlite3

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Loading the model
model = load_model("/content/keras_model.h5", compile=False)

# Loading the labels
class_names = open("/content/labels.txt", "r").readlines()

# Creating the array of the right shape to feed into the keras model
# The 'length' or number of images we can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# path to the image
image_path = "/content/drive/MyDrive/Solution challenge /Data /full/Photo 2023-02-26 02.26.40 PM.jpg"
image = Image.open(image_path).convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.LANCZOS)

# turning the image into a numpy array
image_array = np.asarray(image)

# Normalizing the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Loading the image into the array
data[0] = normalized_image_array

# Predicting the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print the prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

# Store the prediction in SQLite database
conn = sqlite3.connect('predictions.db')
c = conn.cursor()

# Create a table if not exists
c.execute('''CREATE TABLE IF NOT EXISTS predictions
             (image_path TEXT, class_name TEXT, confidence_score REAL)''')

# Insert the prediction into the table
c.execute("INSERT INTO predictions VALUES (?, ?, ?)", (image_path, class_name[2:], confidence_score))

# Commit the changes and close the database
conn.commit()
conn.close()


1/1 [==============================] - 1s 740ms/step
Class: Full
Confidence Score: 0.99686265
